In [1]:
import os
import re
import sys
sys.path.append('./')
from collections import defaultdict
import xml.etree.ElementTree as ET
from xbrl import XBRLParser
from xbrl_parser import XbrlParser
import pandas as pd
import requests

namespaces = {
        'link': 'http://www.xbrl.org/2003/linkbase',
        'xml':'http://www.w3.org/XML/1998/namespace',
        'xlink':'http://www.w3.org/1999/xlink',
        'xsi':'http://www.w3.org/2001/XMLSchema-instance'
        }


base_path = os.getcwd()+'/xbrl_files/'
_dir = 'S100APUV/XBRL/PublicDoc/'
xbrl_filename = base_path+_dir+'jpcrp030000-asr-001_E02066-000_2017-03-31_01_2017-06-29.xbrl'

# get data
xp = XbrlParser(xbrl_filename)

print('getting data...')
xbrl_persed = xp.parse_xbrl(namespaces)
print('done')

df_xbrl_facts = xbrl_persed['facts'] # 金額の定義及び文書情報の定義情報
df_xbrl_labels = xbrl_persed['labels'] # 名称リンク情報
df_xbrl_presentation = xbrl_persed['presentation'] # 表示リンク情報

getting data...
done


In [2]:
df_xbrl_labels.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12956 entries, 0 to 12955
Data columns (total 5 columns):
element_id      12956 non-null object
href            12956 non-null object
label_role      12956 non-null object
label_string    12956 non-null object
lang            12956 non-null object
dtypes: object(5)
memory usage: 506.2+ KB


In [3]:
# extract labels data
df_xbrl_labels = xp.extract_target_data(df_xbrl_labels, lang='ja')
df_xbrl_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12956 entries, 0 to 12955
Data columns (total 5 columns):
element_id      12956 non-null object
href            12956 non-null object
label_role      12956 non-null object
label_string    12956 non-null object
lang            12956 non-null object
dtypes: object(5)
memory usage: 607.3+ KB


In [4]:
# De-duplication of labels data
df_xbrl_labels = df_xbrl_labels.drop_duplicates()
df_xbrl_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12956 entries, 0 to 12955
Data columns (total 5 columns):
element_id      12956 non-null object
href            12956 non-null object
label_role      12956 non-null object
label_string    12956 non-null object
lang            12956 non-null object
dtypes: object(5)
memory usage: 607.3+ KB


In [5]:
dat_fi = pd.merge(df_xbrl_labels, df_xbrl_facts, on='element_id', how='inner')
# specify duration
dat_fi_cyi = dat_fi.loc[dat_fi.context_ref == 'CurrentYearInstant'] # 当期 時点
dat_fi_cyi

,element_id,href,label_role,label_string,lang,amount,context_ref,decimals,unit_ref
19,jpcrp030000-asr_e02066-000_provisionforsalesdi...,jpcrp030000-asr-001_E02066-000_2017-03-31_01_2...,http://www.xbrl.org/2003/role/label,売上値引等引当金,ja,1531910000,CurrentYearInstant,-3,JPY
23,jpcrp030000-asr_e02066-000_provisionforsalesdi...,jpcrp030000-asr-001_E02066-000_2017-03-31_01_2...,http://www.xbrl.org/2003/role/verboseLabel,売上値引等引当金、流動負債,ja,1531910000,CurrentYearInstant,-3,JPY
314,jpcrp_cor_netassetssummaryofbusinessresults,../jpcrp_cor_2017-02-28.xsd#jpcrp_cor_NetAsset...,http://www.xbrl.org/2003/role/label,純資産額,ja,29116630000,CurrentYearInstant,-3,JPY
324,jpcrp_cor_netassetssummaryofbusinessresults,../jpcrp_cor_2017-02-28.xsd#jpcrp_cor_NetAsset...,http://www.xbrl.org/2003/role/verboseLabel,純資産額、経営指標等,ja,29116630000,CurrentYearInstant,-3,JPY
334,jpcrp_cor_totalassetssummaryofbusinessresults,../jpcrp_cor_2017-02-28.xsd#jpcrp_cor_TotalAss...,http://www.xbrl.org/2003/role/label,総資産額,ja,64615799000,CurrentYearInstant,-3,JPY
344,jpcrp_cor_totalassetssummaryofbusinessresults,../jpcrp_cor_2017-02-28.xsd#jpcrp_cor_TotalAss...,http://www.xbrl.org/2003/role/verboseLabel,総資産額、経営指標等,ja,64615799000,CurrentYearInstant,-3,JPY
354,jpcrp_cor_netassetspersharesummaryofbusinessre...,../jpcrp_cor_2017-02-28.xsd#jpcrp_cor_NetAsset...,http://www.xbrl.org/2003/role/label,１株当たり純資産額,ja,724.92,CurrentYearInstant,2,JPYPerShares
364,jpcrp_cor_netassetspersharesummaryofbusinessre...,../jpcrp_cor_2017-02-28.xsd#jpcrp_cor_NetAsset...,http://www.xbrl.org/2003/role/verboseLabel,１株当たり純資産額、経営指標等,ja,724.92,CurrentYearInstant,2,JPYPerShares
414,jpcrp_cor_equitytoassetratiosummaryofbusinessr...,../jpcrp_cor_2017-02-28.xsd#jpcrp_cor_EquityTo...,http://www.xbrl.org/2003/role/label,自己資本比率,ja,0.442,CurrentYearInstant,3,pure
424,jpcrp_cor_equitytoassetratiosummaryofbusinessr...,../jpcrp_cor_2017-02-28.xsd#jpcrp_cor_EquityTo...,http://www.xbrl.org/2003/role/verboseLabel,自己資本比率、経営指標等,ja,0.442,CurrentYearInstant,3,pure


In [6]:
df_xbrl_labels.element_id.ix[df_xbrl_labels.label_string.str
                                          .contains('^純資産合計')].drop_duplicates()

/Users/meganii/.pyenv/versions/3.6.1/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate_ix
  """Entry point for launching an IPython kernel.


12126    jppfs_cor_netassetsmember
12796          jppfs_cor_netassets
Name: element_id, dtype: object

In [7]:
df_xbrl_presentation.role_id.drop_duplicates().values

array([ 'http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_CabinetOfficeOrdinanceOnDisclosureOfCorporateInformationEtcFormNo3AnnualSecuritiesReport',
       'http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_CoverPage',
       'http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_BusinessResultsOfGroup',
       'http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_BusinessResultsOfReportingCompany',
       'http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_MajorShareholders-01',
       'http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_FinancialInformation',
       'http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_NotesSignificantAccountingPoliciesForPreparationOfConsolidatedFinancialStatements',
       'http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_NotesBalanceSheet',
       'http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_NotesConsolidatedBalanceSheet',
       'http://disclosure.edinet-fsa.go.jp/role/jpcrp/rol_NotesStatementOfIncome',
       'http://disclosure.edinet-fsa.go.jp/role/

In [18]:
# rol_StatementOfIncome
# rol_ConsolidatedBalanceSheet
df_xbrl_ps_cbs = df_xbrl_presentation.loc[df_xbrl_presentation.role_id.str
                                             .contains('rol_ConsolidatedBalanceSheet'), :]
df_xbrl_ps_cbs

,arcrole,closed,context_element,from_element_id,from_href,order,preferred_label,role_id,to_element_id,to_href,usable
418,http://www.xbrl.org/2003/arcrole/parent-child,None,None,jppfs_cor_consolidatedbalancesheetheading,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,1.0,http://disclosure.edinet-fsa.go.jp/jppfs/Conso...,http://disclosure.edinet-fsa.go.jp/role/jppfs/...,jppfs_cor_balancesheettable,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,None
419,http://www.xbrl.org/2003/arcrole/parent-child,None,None,jppfs_cor_balancesheettable,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,1.0,None,http://disclosure.edinet-fsa.go.jp/role/jppfs/...,jppfs_cor_consolidatedornonconsolidatedaxis,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,None
420,http://www.xbrl.org/2003/arcrole/parent-child,None,None,jppfs_cor_consolidatedornonconsolidatedaxis,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,1.0,None,http://disclosure.edinet-fsa.go.jp/role/jppfs/...,jppfs_cor_consolidatedmember,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,None
421,http://www.xbrl.org/2003/arcrole/parent-child,None,None,jppfs_cor_consolidatedbalancesheetheading,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,2.0,http://disclosure.edinet-fsa.go.jp/jppfs/Conso...,http://disclosure.edinet-fsa.go.jp/role/jppfs/...,jppfs_cor_balancesheetlineitems,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,None
422,http://www.xbrl.org/2003/arcrole/parent-child,None,None,jppfs_cor_balancesheetlineitems,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,1.0,None,http://disclosure.edinet-fsa.go.jp/role/jppfs/...,jppfs_cor_assetsabstract,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,None
423,http://www.xbrl.org/2003/arcrole/parent-child,None,None,jppfs_cor_assetsabstract,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,1.0,None,http://disclosure.edinet-fsa.go.jp/role/jppfs/...,jppfs_cor_currentassetsabstract,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,None
424,http://www.xbrl.org/2003/arcrole/parent-child,None,None,jppfs_cor_currentassetsabstract,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,1.0,None,http://disclosure.edinet-fsa.go.jp/role/jppfs/...,jppfs_cor_cashanddeposits,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,None
425,http://www.xbrl.org/2003/arcrole/parent-child,None,None,jppfs_cor_currentassetsabstract,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,2.0,None,http://disclosure.edinet-fsa.go.jp/role/jppfs/...,jppfs_cor_notesandaccountsreceivabletrade,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,None
426,http://www.xbrl.org/2003/arcrole/parent-child,None,None,jppfs_cor_currentassetsabstract,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,2.5,None,http://disclosure.edinet-fsa.go.jp/role/jppfs/...,jppfs_cor_shortterminvestmentsecurities,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,None
427,http://www.xbrl.org/2003/arcrole/parent-child,None,None,jppfs_cor_currentassetsabstract,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,3.0,None,http://disclosure.edinet-fsa.go.jp/role/jppfs/...,jppfs_cor_merchandiseandfinishedgoods,http://disclosure.edinet-fsa.go.jp/taxonomy/jp...,None


In [9]:
# BSに関連する Element ID一覧
df_xbrl_ps_cbs.info()
df_xbrl_ps_cbs.from_element_id.drop_duplicates()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 56 entries, 669 to 724
Data columns (total 11 columns):
arcrole            56 non-null object
closed             0 non-null object
context_element    0 non-null object
from_element_id    56 non-null object
from_href          56 non-null object
order              56 non-null object
preferred_label    12 non-null object
role_id            56 non-null object
to_element_id      56 non-null object
to_href            56 non-null object
usable             0 non-null object
dtypes: object(11)
memory usage: 5.2+ KB


669             jppfs_cor_statementofincomeheading
670               jppfs_cor_statementofincometable
671    jppfs_cor_consolidatedornonconsolidatedaxis
673           jppfs_cor_statementofincomelineitems
675                  jppfs_cor_costofsalesabstract
690           jppfs_cor_nonoperatingincomeabstract
699         jppfs_cor_nonoperatingexpensesabstract
709          jppfs_cor_extraordinaryincomeabstract
713            jppfs_cor_extraordinarylossabstract
Name: from_element_id, dtype: object

In [10]:
parent = 'jppfs_cor_assetsabstract'
df_descendant = xp.gather_descendant(df_xbrl_ps_cbs, parent).dropna() # delete nan
df_descendant

Series([], Name: to_element_id, dtype: object)

In [17]:
children = df_xbrl_ps_cbs.to_element_id.loc[df_xbrl_ps_cbs.from_element_id == 'jppfs_cor_assetsabstract']

def recursive(array, df, parent_elem_id):
    for val in df.to_element_id.loc[df.from_element_id == parent_elem_id].values:
        array.append(val)
        recursive(array, df, val)

array = []
recursive(array, df_xbrl_ps_cbs, 'jppfs_cor_costofsalesabstract')
array


['jppfs_cor_beginningmerchandiseandfinishedgoodscos',
 'jppfs_cor_costofpurchasedmerchandiseandfinishedgoodscos',
 'jppfs_cor_costofproductsmanufactured',
 'jpcrp030000-asr_e02066-000_costofsalessubtotalcos',
 'jppfs_cor_transfertootheraccountcos',
 'jppfs_cor_endingmerchandiseandfinishedgoodscos',
 'jpcrp030000-asr_e02066-000_valuationlossongoodsfinishedgoods',
 'jppfs_cor_costofsales']

In [12]:
children = xp.gather_r(df_xbrl_ps_cbs, parent)
children

Series([], Name: to_element_id, dtype: object)

In [13]:
children = df_xbrl_ps_cbs.to_element_id.loc[df_xbrl_ps_cbs.from_element_id == 'jppfs_cor_assetsabstract']
children.apply(lambda x: children.append(df_xbrl_ps_cbs.to_element_id.loc[df_xbrl_ps_cbs.from_element_id == x]))

Series([], Name: to_element_id, dtype: object)

In [14]:
result = None
for label_id in df_descendant.values:
    print(label_id)

In [15]:
label = "jppfs_cor_currentassets"
query = "(label_role == 'http://www.xbrl.org/2003/role/label') and (element_id == " + "'" + label + "')"
df_fi_cyi_caa = dat_fi_cyi.query(query)
df_fi_cyi_caa.drop_duplicates()

,element_id,href,label_role,label_string,lang,amount,context_ref,decimals,unit_ref
930,jppfs_cor_currentassets,../jppfs_cor_2017-02-28.xsd#jppfs_cor_CurrentA...,http://www.xbrl.org/2003/role/label,流動資産,ja,54207971000,CurrentYearInstant,-3,JPY


In [16]:
def recursive(array, df, parent_elem_id):
    for val in df.to_element_id.loc[df.from_element_id == parent_elem_id].values:
        array.append(val)
        recursive(array, df, val)

array = []
recursive(array, df_xbrl_ps_cbs, 'jppfs_cor_assetsabstract')


df_fi_cyi_caa = xp.get_specific_account_name(dat_fi_cyi, array).query("label_role == 'http://www.xbrl.org/2003/role/label'").drop_duplicates()
print(df_fi_cyi_caa[['label_string', 'amount']])

AttributeError: 'NoneType' object has no attribute 'query'

In [ ]:
xp.get_hicharts_json(df_xbrl_ps_cbs, dat_fi_cyi)

In [ ]:
currentassets = []
xp.recursive(currentassets, df_xbrl_ps_cbs, 'jppfs_cor_currentassetsabstract')
currentassets_coa = xp.get_specific_account_name(dat_fi_cyi, currentassets).query("label_role == 'http://www.xbrl.org/2003/role/label'")
for key, value in currentassets_coa[['label_string', 'amount']].values:
    print({
        'name': key,
        'data': [value, None],
        'stack': 'Credit'
    })

In [ ]:
from decimal import Decimal
import json

series = []

currentassets = []
xp.recursive(currentassets, df_xbrl_ps_cbs, 'jppfs_cor_currentassetsabstract')
currentassets_coa = xp.get_specific_account_name(dat_fi_cyi, currentassets).query("label_role == 'http://www.xbrl.org/2003/role/label'")
for key, value in currentassets_coa[['label_string', 'amount']].values:
    series.append({
        'name': key,
        'data': [int(value), None],
        'stack': 'Credit'
    })

noncurrentassets = []
xp.recursive(noncurrentassets, df_xbrl_ps_cbs, 'jppfs_cor_noncurrentassetsabstract')
noncurrentassets_coa = xp.get_specific_account_name(dat_fi_cyi, noncurrentassets).query("label_role == 'http://www.xbrl.org/2003/role/label'")
for key, value in noncurrentassets_coa[['label_string', 'amount']].values:
    series.append({
        'name': key,
        'data': [int(value), None],
        'stack': 'Credit'
    })


liabilities = []
xp.recursive(liabilities, df_xbrl_ps_cbs, 'jppfs_cor_liabilitiesabstract')
liabilities_coa = xp.get_specific_account_name(dat_fi_cyi, liabilities).query("label_role == 'http://www.xbrl.org/2003/role/label'")
for key, value in liabilities_coa[['label_string', 'amount']].values:
    series.append({
        'name': key,
        'data': [int(value), None],
        'stack': 'Debit'
    })

netassets = []
xp.recursive(netassets, df_xbrl_ps_cbs, 'jppfs_cor_netassetsabstract')
netassets_coa = xp.get_specific_account_name(dat_fi_cyi, netassets).query("label_role == 'http://www.xbrl.org/2003/role/label'")
for key, value in netassets_coa[['label_string', 'amount']].values:
    series.append({
        'name': key,
        'data': [int(value), None],
        'stack': 'Debit'
    })
print(json.dumps({"series": series}, ensure_ascii=False))

In [ ]:
Decimal('10').to_integral_exact

In [ ]:
print(int('10'))

In [ ]:
dir(Decimal(10))

In [20]:
# rol_StatementOfIncome
# rol_ConsolidatedBalanceSheet
df_xbrl_ps_soi = df_xbrl_presentation.loc[df_xbrl_presentation.role_id.str
                                             .contains('rol_StatementOfIncome'), :]
df_xbrl_ps_soi.from_element_id.drop_duplicates()

669             jppfs_cor_statementofincomeheading
670               jppfs_cor_statementofincometable
671    jppfs_cor_consolidatedornonconsolidatedaxis
673           jppfs_cor_statementofincomelineitems
675                  jppfs_cor_costofsalesabstract
690           jppfs_cor_nonoperatingincomeabstract
699         jppfs_cor_nonoperatingexpensesabstract
709          jppfs_cor_extraordinaryincomeabstract
713            jppfs_cor_extraordinarylossabstract
Name: from_element_id, dtype: object

In [30]:
dat_fi_cyi = dat_fi.loc[dat_fi.context_ref == 'CurrentYearDuration'] # 当期 時点
dat_fi_cyi

,element_id,href,label_role,label_string,lang,amount,context_ref,decimals,unit_ref
5,jpcrp030000-asr_e02066-000_increasedecreaseinp...,jpcrp030000-asr-001_E02066-000_2017-03-31_01_2...,http://www.xbrl.org/2003/role/label,売上値引等引当金の増減額（△は減少）,ja,-119864000,CurrentYearDuration,-3,JPY
7,jpcrp030000-asr_e02066-000_increasedecreaseinp...,jpcrp030000-asr-001_E02066-000_2017-03-31_01_2...,http://www.xbrl.org/2003/role/verboseLabel,売上値引等引当金の増減額（△は減少）、営業活動によるキャッシュ・フロー,ja,-119864000,CurrentYearDuration,-3,JPY
8,jpcrp030000-asr_e02066-000_notesregardinglosso...,jpcrp030000-asr-001_E02066-000_2017-03-31_01_2...,http://www.xbrl.org/2003/role/label,事業整理損の注記,ja,"<p style=""margin-left: 12px; text-align: left...",CurrentYearDuration,None,None
9,jpcrp030000-asr_e02066-000_notesregardinglosso...,jpcrp030000-asr-001_E02066-000_2017-03-31_01_2...,http://www.xbrl.org/2003/role/verboseLabel,事業整理損の注記 [テキストブロック],ja,"<p style=""margin-left: 12px; text-align: left...",CurrentYearDuration,None,None
10,jpcrp030000-asr_e02066-000_notesregardingnetsa...,jpcrp030000-asr-001_E02066-000_2017-03-31_01_2...,http://www.xbrl.org/2003/role/label,売上高に関する注記,ja,"<p style=""margin-left: 36px; text-align: left...",CurrentYearDuration,None,None
12,jpcrp030000-asr_e02066-000_notesregardingnetsa...,jpcrp030000-asr-001_E02066-000_2017-03-31_01_2...,http://www.xbrl.org/2003/role/verboseLabel,売上高に関する注記 [テキストブロック],ja,"<p style=""margin-left: 36px; text-align: left...",CurrentYearDuration,None,None
128,jpcrp_cor_consolidatedbalancesheettextblock,../jpcrp_cor_2017-02-28.xsd#jpcrp_cor_Consolid...,http://www.xbrl.org/2003/role/label,連結貸借対照表,ja,"<h5>①【連結貸借対照表】</h5> <div> <table style=""table...",CurrentYearDuration,None,None
129,jpcrp_cor_consolidatedbalancesheettextblock,../jpcrp_cor_2017-02-28.xsd#jpcrp_cor_Consolid...,http://www.xbrl.org/2003/role/verboseLabel,連結貸借対照表 [テキストブロック],ja,"<h5>①【連結貸借対照表】</h5> <div> <table style=""table...",CurrentYearDuration,None,None
130,jpcrp_cor_consolidatedstatementofincometextblock,../jpcrp_cor_2017-02-28.xsd#jpcrp_cor_Consolid...,http://www.xbrl.org/2003/role/label,連結損益計算書,ja,"<h6>【連結損益計算書】</h6> <div> <table style=""table-...",CurrentYearDuration,None,None
131,jpcrp_cor_consolidatedstatementofincometextblock,../jpcrp_cor_2017-02-28.xsd#jpcrp_cor_Consolid...,http://www.xbrl.org/2003/role/verboseLabel,連結損益計算書 [テキストブロック],ja,"<h6>【連結損益計算書】</h6> <div> <table style=""table-...",CurrentYearDuration,None,None


In [37]:
array = []
xp.recursive(array, df_xbrl_ps_soi, 'jppfs_cor_statementofincomelineitems')
array
coa = xp.get_specific_account_name(dat_fi_cyi, array).query("label_role == 'http://www.xbrl.org/2003/role/label'")
coa

,element_id,href,label_role,label_string,lang,amount,context_ref,decimals,unit_ref
0,jppfs_cor_netsales,../jppfs_cor_2017-02-28.xsd#jppfs_cor_NetSales,http://www.xbrl.org/2003/role/label,売上高,ja,81842949000,CurrentYearDuration,-3,JPY
3,jppfs_cor_costofsales,../jppfs_cor_2017-02-28.xsd#jppfs_cor_CostOfSales,http://www.xbrl.org/2003/role/label,売上原価,ja,54282372000,CurrentYearDuration,-3,JPY
6,jppfs_cor_grossprofit,../jppfs_cor_2017-02-28.xsd#jppfs_cor_GrossProfit,http://www.xbrl.org/2003/role/label,売上総利益又は売上総損失（△）,ja,27560576000,CurrentYearDuration,-3,JPY
11,jppfs_cor_reversalofprovisionforsalesreturnsgp,../jppfs_cor_2017-02-28.xsd#jppfs_cor_Reversal...,http://www.xbrl.org/2003/role/label,返品調整引当金戻入額,ja,235000000,CurrentYearDuration,-3,JPY
13,jppfs_cor_grossprofitnetgp,../jppfs_cor_2017-02-28.xsd#jppfs_cor_GrossPro...,http://www.xbrl.org/2003/role/label,差引売上総利益又は差引売上総損失（△）,ja,27795576000,CurrentYearDuration,-3,JPY
17,jppfs_cor_sellinggeneralandadministrativeexpenses,../jppfs_cor_2017-02-28.xsd#jppfs_cor_SellingG...,http://www.xbrl.org/2003/role/label,販売費及び一般管理費,ja,17758790000,CurrentYearDuration,-3,JPY
22,jppfs_cor_operatingincome,../jppfs_cor_2017-02-28.xsd#jppfs_cor_Operatin...,http://www.xbrl.org/2003/role/label,営業利益又は営業損失（△）,ja,10036785000,CurrentYearDuration,-3,JPY
29,jppfs_cor_interestincomenoi,../jppfs_cor_2017-02-28.xsd#jppfs_cor_Interest...,http://www.xbrl.org/2003/role/label,受取利息,ja,15062000,CurrentYearDuration,-3,JPY
31,jppfs_cor_dividendsincomenoi,../jppfs_cor_2017-02-28.xsd#jppfs_cor_Dividend...,http://www.xbrl.org/2003/role/label,受取配当金,ja,8101000,CurrentYearDuration,-3,JPY
33,jppfs_cor_purchasediscountsnoi,../jppfs_cor_2017-02-28.xsd#jppfs_cor_Purchase...,http://www.xbrl.org/2003/role/label,仕入割引,ja,1929000,CurrentYearDuration,-3,JPY


In [44]:
import json

series = []

jppfs_cor_netsales = []
xp.recursive(jppfs_cor_netsales, df_xbrl_ps_soi, 'jppfs_cor_statementofincomelineitems')
jppfs_cor_netsales_coa = xp.get_specific_account_name(dat_fi_cyi, jppfs_cor_netsales).query("label_role == 'http://www.xbrl.org/2003/role/label'")
for key, value in jppfs_cor_netsales_coa[['label_string', 'amount']].values:
    series.append({
        'name': key,
        'data': [None, int(value)],
        'stack': 'Credit'
    })
print(json.dumps({"series": series}, ensure_ascii=False))

{"series": [{"name": "売上高", "data": [null, 81842949000], "stack": "Credit"}, {"name": "売上原価", "data": [null, 54282372000], "stack": "Credit"}, {"name": "売上総利益又は売上総損失（△）", "data": [null, 27560576000], "stack": "Credit"}, {"name": "返品調整引当金戻入額", "data": [null, 235000000], "stack": "Credit"}, {"name": "差引売上総利益又は差引売上総損失（△）", "data": [null, 27795576000], "stack": "Credit"}, {"name": "販売費及び一般管理費", "data": [null, 17758790000], "stack": "Credit"}, {"name": "営業利益又は営業損失（△）", "data": [null, 10036785000], "stack": "Credit"}, {"name": "受取利息", "data": [null, 15062000], "stack": "Credit"}, {"name": "受取配当金", "data": [null, 8101000], "stack": "Credit"}, {"name": "仕入割引", "data": [null, 1929000], "stack": "Credit"}, {"name": "受取補償金", "data": [null, 2951000], "stack": "Credit"}, {"name": "その他", "data": [null, 34185000], "stack": "Credit"}, {"name": "営業外収益", "data": [null, 62230000], "stack": "Credit"}, {"name": "支払利息", "data": [null, 15204000], "stack": "Credit"}, {"name": "売上割引", "data": [null, 700505000]